In [22]:
#Ejecutarse para instalar seaborn
%pip install seaborn
%pip install nbformat


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
# Celda 1: Imports y configuración básica
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
import io

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)


In [41]:
import requests

url = "https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/get_analysis"

r = requests.get(url, timeout=20)
print("Código de respuesta:", r.status_code)
print("Tipo de contenido:", r.headers.get("content-type"))
print("Primeros 300 caracteres de la respuesta:")
print(r.text[:300])


Código de respuesta: 200
Tipo de contenido: application/json
Primeros 300 caracteres de la respuesta:
[]


In [24]:
# Celda 2: Funciones para cargar datos (API o CSV local)
API_URL = "https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/get_analysis"

def load_from_api(year=None, province=None, contract_type=None, params_extra=None, timeout=20):
    params = {}
    if year: params['year'] = year
    if province: params['region'] = province
    if contract_type: params['type'] = contract_type
    if params_extra:
        params.update(params_extra)
    resp = requests.get(API_URL, params=params, timeout=timeout)
    resp.raise_for_status()
    data = resp.json()
    # suponiendo que la respuesta trae una lista de registros
    df = pd.DataFrame(data)
    return df

def load_from_csv(path):
    df = pd.read_csv(path)
    return df

# Función de fallback: si API falla, crea un dataframe ejemplo (pequeño) para desarrollo
def sample_dataframe():
    data = {
        'date': pd.date_range('2023-01-01', periods=12, freq='ME'),
        'year': [2023]*12,
        'month': list(range(1,13)),
        'province': ['Azuay','Azuay','Pichincha','Pichincha','Guayas','Guayas','Azuay','Pichincha','Guayas','Azuay','Pichincha','Guayas'],
        'internal_type': ['Compra','Contratación directa','Compra','Licitación','Compra','Compra','Licitación','Contratación directa','Compra','Compra','Licitación','Contratación directa'],
        'contracts': np.random.randint(1,20,12),
        'total': np.random.randint(1000,200000,12)
    }
    return pd.DataFrame(data)


In [25]:
# Celda 3: Cargar datos (elige método)
USE_API = True   # Cambia a True si quieres intentar la API en vivo
CSV_PATH = "compras_publicas.csv"  # si tienes CSV local

try:
    if USE_API:
        df = load_from_api(year=2025)
    else:
        try:
            df = load_from_csv(CSV_PATH)
        except FileNotFoundError:
            df = sample_dataframe()
    print("Datos cargados. Shape:", df.shape)
    display(df.head())
except Exception as e:
    print("Error cargando datos:", e)
    df = sample_dataframe()
    print("Usando dataframe de ejemplo.")
    display(df.head())



Datos cargados. Shape: (195, 4)


,month,internal_type,total,contracts
0,1,Bienes y Servicios únicos,119992.390000,6
1,1,Catálogo electrónico - Compra directa,3602488.254630,454
2,1,Catálogo electrónico - Gran compra mejor oferta,917598.800000,4
3,1,Catálogo electrónico - Mejor oferta,213849.004945,1401
4,1,Comunicación Social – Contratación Directa,88051.520000,6


In [26]:
# Celda 4: Inspección inicial y validación de estructura
print(df.info())
print("\nNulos por columna:")
print(df.isna().sum())
print("\nDuplicados:", df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   month          195 non-null    int64 
 1   internal_type  185 non-null    object
 2   total          180 non-null    object
 3   contracts      195 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 6.2+ KB
None

Nulos por columna:
month             0
internal_type    10
total            15
contracts         0
dtype: int64

Duplicados: 0


In [27]:
# Celda 5: Normalización de nombres de columnas y tipos
# Mapear nombres comunes según la guía
rename_map = {
    'provincia': 'province', 'region': 'province',
    'tipo_contratacion': 'internal_type', 'monto_total': 'total',
    'fecha': 'date', 'cantidad': 'contracts'
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# Asegurar columnas mínimas
expected_cols = ['date', 'year', 'month', 'province', 'internal_type', 'contracts', 'total']
for c in expected_cols:
    if c not in df.columns:
        print(f"Advertencia: columna '{c}' no encontrada. Se creará/intentará derivarla si es posible.")

# Convertir tipos
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
elif 'year' in df.columns and 'month' in df.columns:
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01', errors='coerce')

if 'total' in df.columns:
    df['total'] = pd.to_numeric(df['total'], errors='coerce')
if 'contracts' in df.columns:
    df['contracts'] = pd.to_numeric(df['contracts'], errors='coerce')

# Crear year/month si faltan
if 'year' not in df.columns and 'date' in df.columns:
    df['year'] = df['date'].dt.year
if 'month' not in df.columns and 'date' in df.columns:
    df['month'] = df['date'].dt.month

# Tratar nulos en columnas críticas
df = df.dropna(subset=['total'])  # si total es crítico, eliminamos

if 'internal_type' in df.columns:
    df['internal_type'] = df['internal_type'].fillna('Desconocido')
if 'province' in df.columns:
    df['province'] = df['province'].fillna('Desconocido')

print("Después de normalizar:")
print(df.dtypes)
display(df.head())



Advertencia: columna 'date' no encontrada. Se creará/intentará derivarla si es posible.
Advertencia: columna 'year' no encontrada. Se creará/intentará derivarla si es posible.
Advertencia: columna 'province' no encontrada. Se creará/intentará derivarla si es posible.
Después de normalizar:
month              int64
internal_type     object
total            float64
contracts          int64
dtype: object


,month,internal_type,total,contracts
0,1,Bienes y Servicios únicos,1.199924e+05,6
1,1,Catálogo electrónico - Compra directa,3.602488e+06,454
2,1,Catálogo electrónico - Gran compra mejor oferta,9.175988e+05,4
3,1,Catálogo electrónico - Mejor oferta,2.138490e+05,1401
4,1,Comunicación Social – Contratación Directa,8.805152e+04,6


In [28]:
# Celda 6: Eliminación de duplicados
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Duplicados eliminados: {before-after}")


Duplicados eliminados: 0


In [29]:
# Celda 7: Estadísticas descriptivas y KPIs
total_registros = len(df)
monto_total = df['total'].sum()
promedio = df['total'].mean()
maximo = df['total'].max()
minimo = df['total'].min()

kpis = {
    'Total registros': total_registros,
    'Monto total (USD)': monto_total,
    'Promedio por registro': promedio,
    'Máximo': maximo,
    'Mínimo': minimo
}
kpis


{'Total registros': 180,
 'Monto total (USD)': np.float64(3376235547.1780243),
 'Promedio por registro': np.float64(18756864.150989022),
 'Máximo': np.float64(187357731.29),
 'Mínimo': np.float64(0.0)}

In [30]:
# Celda 8: Describe numérico
display(df[['total','contracts']].describe())


,total,contracts
count,1.800000e+02,180.000000
mean,1.875686e+07,643.155556
std,3.209426e+07,1836.558555
min,0.000000e+00,1.000000
25%,1.334277e+06,10.750000
50%,4.431234e+06,44.000000
75%,1.966145e+07,175.000000
max,1.873577e+08,12048.000000


In [31]:
# Celda 9: Conteos por categoría (top)
print("Conteo por tipo de contratación:")
display(df['internal_type'].value_counts().head(20))

print("\nConteo por provincia:")
if 'province' in df.columns:
    display(df['province'].value_counts().head(20))
else:
    print("Columna 'province' no disponible en este dataset.")


Conteo por tipo de contratación:


internal_type
Bienes y Servicios únicos                                10
Catálogo electrónico - Compra directa                    10
Catálogo electrónico - Gran compra mejor oferta          10
Catálogo electrónico - Mejor oferta                      10
Comunicación Social – Contratación Directa               10
Contratos entre Entidades Públicas o sus subsidiarias    10
Repuestos o Accesorios                                   10
Obra artística, científica o literaria                   10
Subasta Inversa Electrónica                              10
Asesoría y Patrocinio Jurídico                            9
Licitación de Seguros                                     9
Licitación                                                9
Catálogo electrónico - Gran compra puja                   9
Concurso publico                                          8
Contratacion directa                                      7
Lista corta                                               7
Cotización                


Conteo por provincia:
Columna 'province' no disponible en este dataset.


In [32]:
# Celda 10: Total de montos por tipo de contratación (barra)
agg = df.groupby('internal_type', as_index=False)['total'].sum().sort_values('total', ascending=False)
fig = px.bar(agg, x='internal_type', y='total', title='Total de montos por tipo de contratación', labels={'internal_type':'Tipo','total':'Monto total'})
fig.show()


In [33]:
# Celda 11: Evolución mensual de montos totales (series temporales)
if 'date' in df.columns:
    monthly = df.set_index('date').resample('M')['total'].sum().reset_index()
    fig = px.line(monthly, x='date', y='total', title='Evolución mensual de montos totales', labels={'date':'Fecha','total':'Monto total'})
    fig.show()
else:
    monthly = df.groupby('month', as_index=False)['total'].sum().sort_values('month')
    fig = px.line(monthly, x='month', y='total', title='Evolución por mes (sin fecha completa)')
    fig.show()


In [34]:
# Celda 12: Barras apiladas: tipo x mes
if 'month' in df.columns:
    pivot = df.groupby(['month','internal_type'])['total'].sum().reset_index()
    fig = px.bar(pivot, x='month', y='total', color='internal_type', title='Total por tipo de contratación por mes', labels={'month':'Mes','total':'Monto'})
    fig.update_layout(barmode='stack')
    fig.show()


In [35]:
# Celda 13: Pie: proporción de contratos por tipo
counts = df['internal_type'].value_counts().reset_index()
counts.columns = ['internal_type','count']
fig = px.pie(counts, names='internal_type', values='count', title='Proporción de contratos por tipo de contratación')
fig.show()


In [36]:
# Celda 14: Dispersión: contratos vs total
hover_cols = ['month']
if 'province' in df.columns:
    hover_cols.append('province')
if 'date' in df.columns:
    hover_cols.append('date')

fig = px.scatter(
    df,
    x='contracts',
    y='total',
    color='internal_type',
    hover_data=hover_cols,
    title='Monto total vs. cantidad de contratos'
)
fig.show()


In [37]:
# Celda 15: Heatmap año x mes (si hay año y month)
if 'year' in df.columns and 'month' in df.columns:
    pivot = df.groupby(['year','month'])['total'].sum().reset_index()
    table = pivot.pivot(index='year', columns='month', values='total').fillna(0)
    fig = px.imshow(table, labels=dict(x="Mes", y="Año", color="Monto total"), title='Heatmap Año x Mes (monto)')
    fig.show()


In [38]:
# Celda 16: Correlaciones numéricas
corr = df[['total','contracts']].corr()
print("Correlación entre variables numéricas:")
display(corr)
# Scatter matrix (opcional)
# fig = px.scatter_matrix(df[['total','contracts']])
# fig.show()


Correlación entre variables numéricas:


,total,contracts
total,1.000000,0.211526
contracts,0.211526,1.000000


In [39]:
# Celda 17: Análisis por año (KPIs por año)

# Paso 1: Derivar 'year' si es posible
if 'date' in df.columns:
    df['year'] = pd.to_datetime(df['date'], errors='coerce').dt.year
elif 'month' in df.columns:
    df['year'] = 2025  # Asumimos que todos los datos son del mismo año si no hay 'date'

# Paso 2: Validar existencia de 'year' antes de agrupar
if 'year' in df.columns:
    kpi_por_anyo = df.groupby('year').agg(
        total_monto=('total', 'sum'),
        registros=('total', 'count'),
        promedio=('total', 'mean')
    ).reset_index()

    display(kpi_por_anyo)

    # Paso 3: Visualización
    fig = px.bar(
        kpi_por_anyo,
        x='year',
        y='total_monto',
        title='Monto total por año',
        labels={'year': 'Año', 'total_monto': 'Monto Total'}
    )
    fig.show()
else:
    print("⚠️ La columna 'year' no está disponible en este dataset.")




,year,total_monto,registros,promedio
0,2025,3.376236e+09,180,1.875686e+07


In [40]:
# Celda 18: Exportar datos procesados
processed_csv = "processed_compras_publicas.csv"
df.to_csv(processed_csv, index=False)
print("Datos procesados guardados en:", processed_csv)


Datos procesados guardados en: processed_compras_publicas.csv
